#Вебинар 5. Ранжирование и Гибридные рекоммендательные системы

In [1]:
!pip install implicit==0.4.4

     |████████████████████████████████| 1.1 MB 7.8 MB/s 
  Created wheel for implicit: filename=implicit-0.4.4-cp37-cp37m-linux_x86_64.whl size=3413754 sha256=50f9719ec653008709b4e37e91a63cbf1136175c83b806e56d93892a460b89cf
  Stored in directory: /root/.cache/pip/wheels/44/7e/7d/a17324ea207cfbe76aca878b5b8ca0aa932cf55d163329be37
Successfully built implicit


In [2]:
!pip install lightfm

     |████████████████████████████████| 310 kB 6.6 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705370 sha256=589d217478b3a713cc0784d20f381180ce218dfa0836983cb6bc6209619dd178
  Stored in directory: /root/.cache/pip/wheels/f8/56/28/5772a3bd3413d65f03aa452190b00898b680b10028a1021914
Successfully built lightfm


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM

In [4]:
from lightfm.evaluation import precision_at_k, recall_at_k

from src.metrics import precision_at_k as custom_precision, recall_at_k as custom_recall
from src.utils import prefilter_items

In [6]:
data = pd.read_csv('retail_train.csv')

item_features = pd.read_csv('product.csv')
user_features = pd.read_csv('hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1.0,1004906.0,1.0,1.39,364.0,-0.6,1631.0,1.0,0.0,0.0
1,2375,26984851472,1.0,1033142.0,1.0,0.82,364.0,0.0,1631.0,1.0,0.0,0.0


In [7]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[856942.0, 860548.0, 925987.0, 934369.0, 95356..."
1,2,"[846833.0, 978318.0, 899824.0, 901606.0, 10753..."


In [8]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [9]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


#Filter items

In [10]:
data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1.0,1004906.0,1.0,1.39,364.0,-0.6,1631.0,1.0,0.0,0.0
1,2375,26984851472,1.0,1033142.0,1.0,0.82,364.0,0.0,1631.0,1.0,0.0,0.0


In [11]:
n_items_before = data_train['item_id'].nunique()

data_train_filtered = prefilter_items(data_train, take_n_popular=5000, item_features=item_features)

n_items_after = data_train_filtered['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 29482 to 5001


/content/src/utils.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


#Prepare data set

 Prepare csr train matrix:

In [12]:
user_item_matrix = pd.pivot_table(data_train_filtered, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

item_id,27978.0,39317.0,40885.0,70302.0,70423.0,71805.0,78986.0,83396.0,88193.0,91119.0,...,10250091.0,10250293.0,10255549.0,10282046.0,10308337.0,10341234.0,10342382.0,10344585.0,10455579.0,12158974.0
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Prepare CSR test matrix:

In [13]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]

test_user_item_matrix = pd.pivot_table(data_test, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = test_user_item_matrix.astype(float) # необходимый тип матрицы для implicit

In [14]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

#Prepare user and item features

In [15]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
user_feat.shape

(2009, 7)

In [17]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

item_feat.head(2)

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
27978.0,69.0,SEAFOOD-PCKGD,Private,SEAFOOD - FROZEN,SEAFOOD-FRZ-RAW FILLETS,14 OZ
39317.0,1420.0,DRUG GM,National,INFANT FORMULA,INFANT FORMULA MILK BASE,13 OZ


In [18]:
item_feat.shape

(5001, 6)

#Encoding features

In [19]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [20]:
user_feat_lightfm.head(2)

,age_desc_19-24,age_desc_25-34,age_desc_35-44,age_desc_45-54,age_desc_55-64,age_desc_65+,marital_status_code_A,marital_status_code_B,marital_status_code_U,income_desc_100-124K,...,hh_comp_desc_Unknown,household_size_desc_1,household_size_desc_2,household_size_desc_3,household_size_desc_4,household_size_desc_5+,kid_category_desc_1,kid_category_desc_2,kid_category_desc_3+,kid_category_desc_None/Unknown
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#Init model

In [21]:
model = LightFM(no_components=40,
                loss='bpr', # "logistic","bpr", "warp"
                learning_rate=0.01, 
                item_alpha=0.4,
                user_alpha=0.1, 
                random_state=42,
                k=5,
                n=15,
                max_sampled=100)

#Train

In [22]:
model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=20, 
          num_threads=20,
          verbose=True) 

Epoch: 100%|██████████| 20/20 [00:05<00:00,  3.79it/s]


#Getting embeddings

users vectors:

In [23]:
user_emb = model.get_user_representations(features=csr_matrix(user_feat_lightfm.values).tocsr())

In [24]:
user_emb[0].shape # biases

(2009,)

In [25]:
user_emb[1].shape # users vectors

(2009, 40)

items vectors:

In [26]:
item_emb = model.get_item_representations(features=csr_matrix(item_feat_lightfm.values).tocsr())

In [27]:
item_emb[0].shape # biases

(5001,)

In [28]:
item_emb[1].shape # items vectors

(5001, 40)

#Evaluation -> Train precision

In [29]:
# мы можем использовать встроенные метрики lightFM

train_precision = precision_at_k(model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

print(f"Train precision {train_precision}")

Train precision 0.17481335997581482


#Predict

In [30]:
# подготавливаемм id для юзеров и товаров в порядке пар user-item
users_ids_row = data_train_filtered['user_id'].apply(lambda x: userid_to_id[x]).values.astype(int)
items_ids_row = data_train_filtered['item_id'].apply(lambda x: itemid_to_id[x]).values.astype(int)

In [31]:
# модель возвращает меру/скор похожести между соответствующим пользователем и товаром
predictions = model.predict(user_ids=users_ids_row,
                            item_ids=items_ids_row,
                            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                            num_threads=10)

In [32]:
# добавляем наш полученный скор в трейн датафрейм
data_train_filtered['score'] = predictions

In [33]:
data_train_filtered.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price,score
7,2375,26984851516,1.0,1085983.0,1.0,2.99,364.0,-0.4,1642.0,1.0,0.0,0.0,2.99,-0.012427
11,1364,26984896261,1.0,842930.0,1.0,2.19,31742.0,0.0,1520.0,1.0,0.0,0.0,2.19,-0.803548


In [34]:
# создаем предикт датафрейм в формате списка товаров
predict_result = data_train_filtered[['user_id','item_id','score']][data_train_filtered.item_id != 999999].drop_duplicates().sort_values(by=['user_id','score'], ascending=False).groupby('user_id')['item_id']. \
            unique().reset_index()

In [35]:
predict_result.head(2)

,user_id,item_id
0,1,"[1055831.0, 1088462.0, 5565840.0, 1078449.0, 1..."
1,2,"[9365106.0, 5567388.0, 5567582.0, 5568489.0, 5..."


In [36]:
# объединяем предикт и тест датасет для подсчета precision
df_result_for_metrics = result.merge(predict_result, on='user_id', how='inner')

In [37]:
df_result_for_metrics.head(2)

,user_id,actual,item_id
0,1,"[856942.0, 860548.0, 925987.0, 934369.0, 95356...","[1055831.0, 1088462.0, 5565840.0, 1078449.0, 1..."
1,2,"[846833.0, 978318.0, 899824.0, 901606.0, 10753...","[9365106.0, 5567388.0, 5567582.0, 5568489.0, 5..."


#Test with custom precision func

In [38]:
precision = df_result_for_metrics.apply(lambda row: custom_precision(row['item_id'], row['actual'],k=5), axis=1).mean()
print(f"Precision: {precision}")

Precision: 0.1531821259309405


#GridSearch of the model(Loss, no_components, regularization)

In [51]:
# GridSearch-Loss
ls = ["bpr", "warp"]
models_list = []

for i in ls:
  model_2 = LightFM(no_components=40,
                  loss=i, 
                  learning_rate=0.01, 
                  item_alpha=0.4,
                  user_alpha=0.1, 
                  random_state=42,
                  k=5,
                  n=15,
                  max_sampled=100)
  
  models_list.append(model_2)

In [52]:
models_list

In [53]:
for models in models_list:
  models.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=20, 
          num_threads=20,
          verbose=True)

Epoch: 100%|██████████| 20/20 [00:06<00:00,  3.33it/s]


In [54]:
# мы можем использовать встроенные метрики lightFM
train_precision_list = []

for mod in models_list:
  train_precision = precision_at_k(mod, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

  train_precision_list.append(train_precision)

print(f"Train precision: {train_precision_list}")

Train precision: [0.17481336, 0.015530115]


Вывод: лучший метод Loss в данной модели это 'bpr'

In [55]:
# GridSearch-no_components
models_list_2 = []

for j in range(10, 100, 10):
  model_3 = LightFM(no_components=j,
                    loss='bpr', 
                    learning_rate=0.01, 
                    item_alpha=0.4,
                    user_alpha=0.1, 
                    random_state=42,
                    k=5,
                    n=15,
                    max_sampled=100)
  
  models_list_2.append(model_3)

In [56]:
models_list_2

In [57]:
for models in models_list_2:
  models.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=20, 
          num_threads=20,
          verbose=True)

Epoch: 100%|██████████| 20/20 [00:11<00:00,  1.76it/s]


In [58]:
# мы можем использовать встроенные метрики lightFM
train_precision_list_2 = []

for mod in models_list_2:
  train_precision = precision_at_k(mod, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

  train_precision_list_2.append(train_precision)

print(f"Train precision: {train_precision_list_2}")

Train precision: [0.1741165, 0.17471382, 0.17421606, 0.17421606, 0.1749129, 0.17461427, 0.17351918, 0.1745147, 0.17381786]


Вывод: лучшее значение 'no_components' данной модели 50

In [59]:
# GridSearch-regularization
lr = [0.01, 0.05, 0.1, 0.25, 0.5, 0.75]
models_list_3 = []

for l in lr:
  model_4 = LightFM(no_components=50,
                    loss='bpr', 
                    learning_rate=l, 
                    item_alpha=0.4,
                    user_alpha=0.1, 
                    random_state=42,
                    k=5,
                    n=15,
                    max_sampled=100)
  
  models_list_3.append(model_4)

In [60]:
models_list_3

In [61]:
for models in models_list_3:
  models.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=20, 
          num_threads=20,
          verbose=True)

Epoch: 100%|██████████| 20/20 [00:06<00:00,  2.93it/s]


In [63]:
# мы можем использовать встроенные метрики lightFM
train_precision_list_3 = []

for mod in models_list_3:
  train_precision = precision_at_k(mod, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

  train_precision_list_3.append(train_precision)

print(f"Train precision: {train_precision_list_3}")

Train precision: [0.1737183, 0.17401695, 0.1737183, 0.17351918, 0.0024888003, 0.0017919363]


Вывод: лучшее значение 'learning_rate' данной модели 0.05